Initialisation

In [24]:
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt

N = 10 #Number of points

Dataset generation

In [25]:
def pointgeneration():
    """
    Return a list of two numbers in [0,1]
    """
    return rd.rand(2)

In [38]:
def generateB(n):
    """
    Return list of n list of 2 elements
    """
    return rd.uniform(size=(n, 2))
    
print(generateB(N))

[[0.12670282 0.22118901]
 [0.83782165 0.17524534]
 [0.00162061 0.75879405]
 [0.74067724 0.66252299]
 [0.45711636 0.82018315]
 [0.08750722 0.42886249]
 [0.55087455 0.3464169 ]
 [0.73148344 0.21078339]
 [0.28147798 0.891996  ]
 [0.23576452 0.79508998]]


In [47]:
def generateA(n):

    A=generateB(n-4)

    #adding the corner
    A= np.append(A,[[0,0]],axis=0)
    A= np.append(A,[[0,1]],axis=0)
    A= np.append(A,[[1,0]],axis=0)
    A= np.append(A,[[1,1]],axis=0)

    #rotating the set
    angle = rd.uniform() * 2 * np.pi
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    A = A @ rotation_matrix

    # Shuffle the points
    np.random.shuffle(A)
    return A


In [55]:
def generateC(n,r):
    C=np.empty((1,2))
    while np.size(C,0)<n:
        x,y=rd.uniform(0,r),rd.uniform(0,r)
        if x**2+y**2<=r**2:
            C = np.append(C,[[x,y]], axis=0)
    return C


In [58]:
def generateD(n,r):
    D=np.empty((1,2))
    for i in range(n):
        angle = rd.uniform()*2*np.pi
        D = np.append(D,[[r*np.cos(angle),r*np.sin(angle)]],axis=0)
    return D
